<a href="https://colab.research.google.com/github/MMaggieZhou/sentiment_analysis/blob/main/sentiment_analysis_bert_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score

import spacy
from scipy.sparse import hstack
import matplotlib.pyplot as plt

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
import re
import unicodedata

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Normalize unicode characters
    text = unicodedata.normalize("NFKC", text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters (optional)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    return text

def load_data(file):
    return pd.read_csv(file, names=['id', 'entity', 'label', 'text']).set_index('id')

def clean_data(df):
    return df.drop_duplicates().dropna()

def drop_label(df):
    return df[df['label'].isin({'Positive','Negative'})]

label_encoder = LabelBinarizer()
def preprocess(df, training):
    if training:
        df['label_num'] = label_encoder.fit_transform(df['label'])
    else:
        df['label_num'] = label_encoder.transform(df['label'])
    df['text_processed'] = df['text'].apply(preprocess_text)
    return clean_data(df)

In [ ]:
training_df = load_data("/content/twitter_training.csv")
training_df = drop_label(clean_data(training_df))
training_df = preprocess(training_df, training=True)
test_df = load_data("/content/twitter_validation.csv")
test_df = drop_label(clean_data(test_df))
test_df = preprocess(test_df, training=False)

In [ ]:
def get_bert_embedding(sentence):
    tokens = tokenizer(sentence, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bert_model.to(device)
    for k,v in tokens.items():
        tokens[k] = v.to(device)
    with torch.no_grad():
        outputs = bert_model(**tokens)
    # Extract the [CLS] token embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return cls_embedding

In [ ]:
X_test = np.array([get_bert_embedding(sent) for sent in test_df["text_processed"]])

In [ ]:
X_train = np.array([get_bert_embedding(sent) for sent in training_df["text_processed"]])

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': LinearSVC(),
}
for model in models.values():
    model.fit(X_train, training_df['label_num'])

In [ ]:
y_probs = {}
y_predicts = {}
for name, model in models.items():
    training_score = model.score(X_train, training_df['label_num'])
    testing_score = model.score(X_test, test_df['label_num'])
    print(f"{name} training score (bias): {training_score * 100:.2f} ; testing score (variance):  {testing_score * 100:.2f}")

    y_probs[name] = model._predict_proba_lr(X_test)[:, 1]
    y_predicts[name] = model.predict(X_test)

LogisticRegression training score (bias): 83.39 ; testing score (variance):  85.64
SVM training score (bias): 83.34 ; testing score (variance):  85.64


In [ ]:
import re
import unicodedata

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Normalize unicode characters
    text = unicodedata.normalize("NFKC", text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters (optional)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    return text
nlp = spacy.load("en_core_web_sm")
def preprocesss_text(text):
    doc = nlp(text)
    ret = []
    for token in doc:
        if token.is_stop:
            continue
        ret.append(token.lemma_)
    return ' '.join(ret)

def load_data(file):
    return pd.read_csv(file, names=['id', 'entity', 'label', 'text']).set_index('id')

def clean_data(df):
    return df.drop_duplicates().dropna()

def drop_label(df):
    return df[df['label'].isin({'Positive','Negative'})]

label_encoder = LabelBinarizer()
def preprocess(df, training):
    if training:
        df['label_num'] = label_encoder.fit_transform(df['label'])
    else:
        df['label_num'] = label_encoder.transform(df['label'])
    df['text_processed'] = df['text'].apply(preprocesss_text)
    return clean_data(df)

In [ ]:
training_df = load_data("/content/twitter_training.csv")
training_df = drop_label(clean_data(training_df))
training_df = preprocess(training_df, training=True)
test_df = load_data("/content/twitter_validation.csv")
test_df = drop_label(clean_data(test_df))
test_df = preprocess(test_df, training=False)

In [ ]:
X_train = np.array([get_bert_embedding(sent) for sent in training_df["text_processed"]])

In [ ]:
X_test = np.array([get_bert_embedding(sent) for sent in test_df["text_processed"]])

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': LinearSVC(),
}
for model in models.values():
    model.fit(X_train, training_df['label_num'])

In [ ]:
y_probs = {}
y_predicts = {}
for name, model in models.items():
    training_score = model.score(X_train, training_df['label_num'])
    testing_score = model.score(X_test, test_df['label_num'])
    print(f"{name} training score (bias): {training_score * 100:.2f} ; testing score (variance):  {testing_score * 100:.2f}")

    y_probs[name] = model._predict_proba_lr(X_test)[:, 1]
    y_predicts[name] = model.predict(X_test)

LogisticRegression training score (bias): 80.30 ; testing score (variance):  79.01
SVM training score (bias): 80.24 ; testing score (variance):  78.64
